In [1]:
!pip install delta-spark

  Using cached pyspark-3.5.5-py2.py3-none-any.whl
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.0
    Can't uninstall 'pyspark'. No files were found to uninstall.


In [2]:
# !pyspark --packages io.delta:delta-spark_2.12:3.2.0 --conf "spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension" --conf "spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog"

In [3]:
# df = sc.textFile.json('/home/matheus/Documentos/teste_engenheiro/data')

In [77]:
from pyspark.sql import SparkSession
import pandas as pd
import requests, json, os

from pyspark.sql import functions as F
from pyspark.sql.types import StructField, StringType, ArrayType, StructType
from delta import *
from pyspark.sql.functions import regexp_replace

In [5]:
spark = SparkSession.builder.appName("DesafioEngenhariaDados") \
        .config("spark.jars", "/home/jovyan/work/postgresql/postgresql-42.7.5.jar") \
        .config("spark.jars.packages", "io.delta:delta-core_2.12:2.0.0") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") \
        .master("local[*]").getOrCreate()

In [6]:
# spark = SparkSession.builder.appName("DesafioEngenhariaDados") \
#         .config("spark.jars", "/home/jovyan/work/postgresql/postgresql-42.7.5.jar") \
#         .config("spark.jars.packages", "io.delta:delta-core_2.12:2.0.0") \
#         .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
#         .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
#         .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") \
#         .master("spark://172.18.0.4:7077").getOrCreate()

In [7]:
spark.sparkContext.getConf().getAll()

[('spark.repl.local.jars',
  'file:///home/jovyan/work/postgresql/postgresql-42.7.5.jar,file:///home/jovyan/.ivy2/jars/io.delta_delta-core_2.12-2.0.0.jar,file:///home/jovyan/.ivy2/jars/io.delta_delta-storage-2.0.0.jar,file:///home/jovyan/.ivy2/jars/org.antlr_antlr4-runtime-4.8.jar,file:///home/jovyan/.ivy2/jars/org.codehaus.jackson_jackson-core-asl-1.9.13.jar'),
 ('spark.submit.pyFiles',
  '/home/jovyan/.ivy2/jars/io.delta_delta-core_2.12-2.0.0.jar,/home/jovyan/.ivy2/jars/io.delta_delta-storage-2.0.0.jar,/home/jovyan/.ivy2/jars/org.antlr_antlr4-runtime-4.8.jar,/home/jovyan/.ivy2/jars/org.codehaus.jackson_jackson-core-asl-1.9.13.jar'),
 ('spark.driver.host', 'd20967de1d61'),
 ('spark.app.name', 'DesafioEngenhariaDados'),
 ('spark.driver.port', '44813'),
 ('spark.executor.id', 'driver'),
 ('spark.app.initial.jar.urls',
  'spark://d20967de1d61:44813/jars/postgresql-42.7.5.jar'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --a

In [8]:
# dfPd = pd.read_json('https://raw.githubusercontent.com/wandersondsm/teste_engenheiro/refs/heads/main/data/Aaron697_Brekke496_2fa15bc7-8866-461a-9000-f739e425860a.json', encoding='utf-8')

In [9]:
# dfPd.head(5)

In [10]:
url = "https://api.github.com/repos/wandersondsm/teste_engenheiro/git/trees/main?recursive=1"

In [11]:
resp = requests.get(url)

lista = []
for repo in resp.json()['tree'][2:-1]:
    # print(repo.get('path'))
    # print(repo.get('url'))
    lista.append("https://raw.githubusercontent.com/wandersondsm/teste_engenheiro/refs/heads/main/" + repo.get('path'))

In [12]:
# Função para baixar um arquivo
def download_file(url, folder):
    try:
        # Extrai o nome do arquivo da URL
        file_name = os.path.join(folder, url.split("/")[-1])
        
        # Faz a requisição HTTP para baixar o arquivo
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Lança exceção se houver erro HTTP
        
        # Salva o arquivo na pasta especificada
        with open(file_name, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        
        print(f"Download concluído: {file_name}")
    
    except requests.exceptions.RequestException as e:
        print(f"Falha ao baixar {url}: {e}")

In [13]:
download_folder = "postgresql"
# Criando a pasta para armazenar o JAR do postgreSQL
if not os.path.exists(download_folder):
    os.makedirs(download_folder)

download_file("https://jdbc.postgresql.org/download/postgresql-42.7.5.jar", download_folder)

Download concluído: postgresql/postgresql-42.7.5.jar


In [ ]:
# Pasta onde os arquivos serão salvos
download_folder = "data"

if not os.path.exists(download_folder):
    os.makedirs(download_folder)

for url in lista:
    download_file(url, download_folder)

In [105]:
def exploded_json_columns(df, column_name_schema):
    json_fields = exploded_df.schema[f'{column_name_schema}'].dataType.fieldNames()
    
    df_dinamico_tratado = exploded_df.select(F.col("type"), *[F.col(f"entry_explode.{field}").alias(field) for field in json_fields])
    return df_dinamico_tratado

In [15]:
# def get_json_keys(schema, prefix):
#     keys = []
#     for field in schema.fields:
#         if isinstance(field.dataType, StructType):
#             if prefix:
#                 new_prefix = f"{prefix}.{field.name}"
#             else:
#                 new_prefix = field.name
#             keys += get_json_keys(field.dataType, new_prefix)
#         elif isinstance(field.dataType, ArrayType) and isinstance(field.dataType.elementType, StructType):
#             if prefix:
#                 new_prefix = f"{prefix}.{field.name}"
#             else:
#                 new_prefix = field.name
#             keys += get_json_keys(field.dataType.elementType, new_prefix)
#         else:
#             if prefix:
#                 keys.append(f"{prefix}.{field.name}")
#             else:
#                 keys.append(field.name)
#     return keys
# cols = get_json_keys(dynamic_schema, "nome")

In [16]:
df = spark.read.option("multiline","true").json('data/*.json')

In [17]:
# df_raw = spark.read.format("parquet").load("/home/jovyan/work/datalake/raw")
# df_raw = spark.read.parquet("/home/jovyan/work/datalake/raw")
# df.write.mode("overwrite").format("parquet").save("/home/jovyan/work/datalake/raw")

In [18]:
# exploded_df = df.withColumn("detail", explode(col("details")))
exploded_df = df.withColumn("entry_explode", F.explode(F.col("entry")))
exploded_df.show(5)

+--------------------+------------+-----------+--------------------+
|               entry|resourceType|       type|       entry_explode|
+--------------------+------------+-----------+--------------------+
|[{urn:uuid:071812...|      Bundle|transaction|{urn:uuid:0718123...|
|[{urn:uuid:071812...|      Bundle|transaction|{urn:uuid:78418ee...|
|[{urn:uuid:071812...|      Bundle|transaction|{urn:uuid:0000016...|
|[{urn:uuid:071812...|      Bundle|transaction|{urn:uuid:2b772e4...|
|[{urn:uuid:071812...|      Bundle|transaction|{urn:uuid:e872482...|
+--------------------+------------+-----------+--------------------+
only showing top 5 rows



In [19]:
json_fields = exploded_df.schema["entry_explode"].dataType.fieldNames()

df_dinamico = exploded_df.select(F.col("type"), *[F.col(f"entry_explode.{field}").alias(field) for field in json_fields])

df_dinamico.show()

+-----------+--------------------+--------------------+--------------------+
|       type|             fullUrl|             request|            resource|
+-----------+--------------------+--------------------+--------------------+
|transaction|urn:uuid:0718123b...|     {POST, Patient}|{NULL, NULL, NULL...|
|transaction|urn:uuid:78418eeb...|{POST, Organization}|{NULL, NULL, true...|
|transaction|urn:uuid:0000016d...|{POST, Practitioner}|{NULL, NULL, true...|
|transaction|urn:uuid:2b772e4b...|   {POST, Encounter}|{NULL, NULL, NULL...|
|transaction|urn:uuid:e8724820...|   {POST, Condition}|{NULL, NULL, NULL...|
|transaction|urn:uuid:eb7f5891...|{POST, Medication...|{NULL, NULL, NULL...|
|transaction|urn:uuid:716c1bde...|       {POST, Claim}|{NULL, NULL, NULL...|
|transaction|urn:uuid:a8b7a2d0...|    {POST, CareTeam}|{NULL, NULL, NULL...|
|transaction|urn:uuid:8dcdd29e...|        {POST, Goal}|{NULL, {[{in-prog...|
|transaction|urn:uuid:9ba7315c...|        {POST, Goal}|{NULL, {[{in-prog...|

In [20]:
# exploded_df.printSchema()

In [21]:
json_fields = df_dinamico.schema["resource"].dataType.fieldNames()

df_dinamico_resource = df_dinamico.select(F.col("type"), *[F.col(f"resource.{field}").alias(field) for field in json_fields])

In [22]:
# df_dinamico.select(['fullUrl', 'request', 'resource']).collect(0)
# df_dinamico_resource.where(df_dinamico_resource.resourceType == "Condition").show(5) 
# df_dinamico_resource.where(df_dinamico_resource.resourceType == "Condition").count() .groupBy("code.text")
# df_dinamico_resource.where((df_dinamico_resource.resourceType == "Condition") & (df_dinamico_resource.code.text == "Cardiac Arrest")).count() #.where(df_dinamico_resource.resourceType == "Condition") 

# Quais são as 10 condições médicas mais comuns?
df_dinamico_resource.where(df_dinamico_resource.resourceType == "Condition").groupBy("code.text").count().sort(F.col("count").desc()).show(10) #.where(df_dinamico_resource.resourceType == "Condition") 

+--------------------+-----+
|                text|count|
+--------------------+-----+
|Viral sinusitis (...| 1237|
|Acute viral phary...|  754|
|Acute bronchitis ...|  548|
|    Normal pregnancy|  516|
|         Prediabetes|  388|
|   Anemia (disorder)|  382|
|Body mass index 3...|  346|
|        Hypertension|  330|
|Chronic sinusitis...|  251|
|Miscarriage in fi...|  208|
+--------------------+-----+
only showing top 10 rows



In [23]:
# Quais são os 10 medicamentos mais prescritos?
df_dinamico_resource.where(df_dinamico_resource.resourceType == "MedicationRequest").groupBy("medicationCodeableConcept.text").count().sort(F.col("count").desc()).show(10) 

+--------------------+-----+
|                text|count|
+--------------------+-----+
|1 ML Epoetin Alfa...| 4133|
|   Simvistatin 10 MG| 2306|
|Cisplatin 50 MG I...|  880|
|PACLitaxel 100 MG...|  820|
|Acetaminophen 325...|  622|
|Amoxicillin 250 M...|  266|
|Naproxen sodium 2...|  264|
|Ibuprofen 200 MG ...|  178|
|Hydrochlorothiazi...|  146|
|Acetaminophen 160...|  144|
+--------------------+-----+
only showing top 10 rows



In [24]:
# Quantos pacientes são do sexo masculino?
df_dinamico_resource.where((df_dinamico_resource.resourceType == "Patient") & (df_dinamico_resource.gender == "male")).groupBy("gender").count().sort(F.col("count").desc()).show(10)

+------+-----+
|gender|count|
+------+-----+
|  male|  567|
+------+-----+



In [25]:
# Quantos pacientes são do sexo masculino e quantos são do sexo feminino?
df_dinamico_resource.where(df_dinamico_resource.resourceType == "Patient").groupBy("gender").count().sort(F.col("count").desc()).show(10)

+------+-----+
|gender|count|
+------+-----+
|female|  613|
|  male|  567|
+------+-----+



In [ ]:
# .alias("total")

In [53]:
condicao_medica = df_dinamico_resource.where(df_dinamico_resource.resourceType == "Condition").groupBy("code.text").count().sort(F.col("count").desc()).withColumnRenamed("text", "condicoes_medicas").withColumnRenamed("count", "total")

In [54]:
condicao_medica.show()

+--------------------+-----+
|   condicoes_medicas|total|
+--------------------+-----+
|Viral sinusitis (...| 1237|
|Acute viral phary...|  754|
|Acute bronchitis ...|  548|
|    Normal pregnancy|  516|
|         Prediabetes|  388|
|   Anemia (disorder)|  382|
|Body mass index 3...|  346|
|        Hypertension|  330|
|Chronic sinusitis...|  251|
|Miscarriage in fi...|  208|
|        Otitis media|  191|
|Streptococcal sor...|  189|
|      Hyperlipidemia|  142|
|     Sprain of ankle|  127|
|            Diabetes|  101|
|Hypertriglyceride...|   97|
|Metabolic syndrom...|   96|
|      Polyp of colon|   95|
|Osteoarthritis of...|   69|
|Sinusitis (disorder)|   68|
+--------------------+-----+
only showing top 20 rows



In [55]:
medicamentos = df_dinamico_resource.where(df_dinamico_resource.resourceType == "MedicationRequest").groupBy("medicationCodeableConcept.text").count().sort(F.col("count")).withColumnRenamed("text", "remedios").withColumnRenamed("count", "total")

In [56]:
sexo_paciente = df_dinamico_resource.where((df_dinamico_resource.resourceType == "Patient") & (df_dinamico_resource.gender == "male")).groupBy("gender").count().sort(F.col("count").desc()).withColumnRenamed("gender", "genero").withColumnRenamed("count", "total")

In [57]:
def grava_tabela(df, nome_tabela):
    df.write.mode("overwrite").format("jdbc").option("url", "jdbc:postgresql://172.18.0.2:5432/desafio") \
        .option("dbtable", f"{nome_tabela}") \
        .option("user", "admin") \
        .option("password", "123@mudar*").option("driver", "org.postgresql.Driver").save()

In [58]:
grava_tabela(condicao_medica, "condicao_medica")
grava_tabela(medicamentos, "medicamentos")
grava_tabela(sexo_paciente, "sexo_paciente")

In [32]:
# .write.mode("overwrite").format("jdbc").option("url", "jdbc:postgresql://172.18.0.2:5432/desafio") \
#         .option("dbtable", "condicao_medica") \
#         .option("user", "admin") \
#         .option("password", "123@mudar*").option("driver", "org.postgresql.Driver").save()

In [259]:
dynamic_schema

StructType([StructField('family', StringType(), True), StructField('given', ArrayType(StringType(), True), True), StructField('prefix', ArrayType(StringType(), True), True), StructField('suffix', ArrayType(StringType(), True), True), StructField('use', StringType(), True)])

In [249]:
patients_json.groupBy("gender").count().sort(F.col("count").desc()).show()

+------+-----+
|gender|count|
+------+-----+
|female|  613|
|  male|  567|
+------+-----+



In [262]:
patients_json = df_dinamico.filter(df_dinamico.resource.resourceType == "Patient").select(
    F.col("resource.id").alias("patient_id"),
    F.col("resource.name").alias('nome'),
    F.col("resource.gender").alias("gender"),
    F.col("resource.birthDate").alias("birth_date")
)

dynamic_schema = spark.read.json(patients_json.rdd.map(lambda row: row.nome)).schema

patients_json = patients_json.withColumn("exploded_name" , F.from_json(F.col("nome"), ArrayType(dynamic_schema))) # .select("exploded_name.*")

# patients_json.show()

patients_json = patients_json.select(
    F.col("patient_id"),
    F.col("exploded_name")[0].family.alias("family"),
    F.col("exploded_name")[0].given.alias("given"),
    F.col("exploded_name")[0].prefix.alias("prefix"),
    F.col("exploded_name")[0].use.alias("use"),
    F.col("gender"),
    F.col("birth_date")
)

patients_json.show()

+--------------------+-------------+-------------+------+--------+------+----------+
|          patient_id|       family|        given|prefix|     use|gender|birth_date|
+--------------------+-------------+-------------+------+--------+------+----------+
|0718123b-5034-496...|     Jerde200|   [Floyd420]| [Mr.]|official|  male|1917-08-06|
|df121e33-f3dc-4d0...|  Leuschke194|[Theodore876]| [Mr.]|official|  male|1924-06-30|
|f156b85a-67c6-423...|     Rojas511|   [Pedro316]| [Mr.]|official|  male|1912-10-17|
|b0f49c80-b59b-4df...|Echevarría842|   [Ester635]|[Mrs.]|official|female|1927-06-20|
|37ff5999-5a5a-472...|  Baumbach677|[Faustino767]| [Mr.]|official|  male|1951-05-24|
|cbaf7927-acab-449...|    Wunsch504|   [Myles862]| [Mr.]|official|  male|1911-04-08|
|32fde49e-97b8-483...| Hettinger594|  [Wilbur107]| [Mr.]|official|  male|1958-03-22|
|543a7d59-fe25-464...|   Murazik203|   [Amado512]| [Mr.]|official|  male|1919-09-19|
|6a92b6f5-0f67-4bd...|     Rohan584|   [Rhett759]| [Mr.]|official

In [201]:
claims_json = df_dinamico.filter(df_dinamico.resource.resourceType == "Claim").select(
    F.col("resource.total").alias('claim_total_value'),
    F.col("resource.id").alias("claim_id"),
    F.col("resource.patient.reference").alias("patient_reference"),
    F.col("resource.provider.reference").alias("organization_reference"),
    F.col("resource.created").alias("claim_created"))

dynamic_schema_claims = spark.read.json(claims_json.rdd.map(lambda row: row.claim_total_value)).schema
claims_json_exploded = claims_json.withColumn("exploded_claim_total_value" , F.from_json(F.col("claim_total_value"), dynamic_schema_claims)) # .select("exploded_name.*")
claims_json_exploded.show()

+--------------------+--------------------+--------------------+----------------------+--------------------+--------------------------+
|   claim_total_value|            claim_id|   patient_reference|organization_reference|       claim_created|exploded_claim_total_value|
+--------------------+--------------------+--------------------+----------------------+--------------------+--------------------------+
|{"value":263.49,"...|716c1bde-4c02-428...|urn:uuid:0718123b...|  urn:uuid:78418eeb...|1935-09-30T09:19:...|             {USD, 263.49}|
|{"value":129.16,"...|0b06c9bc-9ff8-4fc...|urn:uuid:0718123b...|  urn:uuid:78418eeb...|1935-09-30T09:19:...|             {USD, 129.16}|
|{"value":129.16,"...|024d7ca2-3907-4fe...|urn:uuid:0718123b...|  urn:uuid:4861d01f...|1941-12-28T10:04:...|             {USD, 129.16}|
|{"value":129.16,"...|6242e6d4-e30b-453...|urn:uuid:0718123b...|  urn:uuid:4861d01f...|1945-07-31T11:04:...|             {USD, 129.16}|
|{"value":77.49,"c...|cf12ad92-7400-44c...|urn:u

In [203]:
organizations = df_dinamico.filter(df_dinamico.resource.resourceType == "Organization").select(
    F.col("resource.id").alias("organization_id"),
    F.col("resource.name").alias("organization_name"),
    F.col("resource.address")[0].city.alias("city"),
    F.col("resource.address")[0].state.alias("state"),
    F.col("resource.address")[0].country.alias("country")
)
organizations.show()

+--------------------+--------------------+---------------+-----+-------+
|     organization_id|   organization_name|           city|state|country|
+--------------------+--------------------+---------------+-----+-------+
|78418eeb-8ad8-3bc...|           PCP207753|           HULL|   MA|     US|
|4861d01f-019c-3da...|NORTH SHORE MEDIC...|          SALEM|   MA|     US|
|8881e3ee-e915-31b...|Lynn Community Ba...|           Lynn|   MA|     US|
|9b5f23f1-c387-3a7...|      FAMILY DOCTORS|     SWAMPSCOTT|   MA|     US|
|65110f22-e3e1-34f...|             PCP8766|         MEDWAY|   MA|     US|
|24cb4eab-6166-353...|MILFORD REGIONAL ...|        MILFORD|   MA|     US|
|53b25c4b-e95f-3dd...|Framingham Outpat...|     Framingham|   MA|     US|
|b06770b5-b695-3c1...| GENERAL HEALTH CARE|NORTH ATTLEBORO|   MA|     US|
|de36f40b-6ae9-3c9...|            PCP97489|   NORTHBOROUGH|   MA|     US|
|331f4c11-d298-308...|CLINTON HOSPITAL ...|        CLINTON|   MA|     US|
|ebc3f5c4-6700-34a...|      NOBLE HOSP

In [225]:
practitioners = df_dinamico.filter(df_dinamico.resource.resourceType == "Practitioner").select(
    F.col("resource.id").alias("practitioner_id"),
    F.col("resource.name").alias("name"),
    # F.col("resource.name")[0].given[0].alias("practitioner_given_name"),
    F.col("resource.gender").alias("practitioner_gender")
)
# practitioners.show()
dynamic_schema_practitioners = spark.read.json(practitioners.rdd.map(lambda row: row.name)).schema
practitioners_json_exploded = practitioners.withColumn("exploded_name" , F.from_json(F.col("name"), ArrayType(dynamic_schema_practitioners))) # .select("exploded_name.*")
# practitioners_json_exploded.show()

practitioners_json_exploded.select(
    F.col("practitioner_id"),
    # F.col("resource.name").alias("name"),
    F.col("exploded_name")[0].family.alias("practitioner_family_name"),
    F.col("exploded_name")[0].given[0].alias("practitioner_given_name"),
    F.col("practitioner_gender")
).show()

+--------------------+------------------------+-----------------------+-------------------+
|     practitioner_id|practitioner_family_name|practitioner_given_name|practitioner_gender|
+--------------------+------------------------+-----------------------+-------------------+
|0000016d-3a85-4cc...|             Iglesias873|              Víctor493|               male|
|0000016d-3a85-4cc...|               Kohler843|               Bobby524|               male|
|0000016d-3a85-4cc...|              Kessler503|               Wayne846|               male|
|0000016d-3a85-4cc...|                Berge125|             Thurman577|               male|
|0000016d-3a85-4cc...|                 Hand679|                Jane262|             female|
|0000016d-3a85-4cc...|                Nolan344|            Kymberly381|             female|
|0000016d-3a85-4cc...|             Homenick806|              Stevie682|             female|
|0000016d-3a85-4cc...|              Pacocha935|                Mira443|         

In [ ]:
# practitioners_json_exploded.select(F.col("exploded_name")[0].family.alias("practitioner_family_name")).show()

In [ ]:
# practitioners_json_exploded.select(F.col("exploded_name")[0].given[0].alias("practitioner_given_name")).show()

In [227]:
encounters = df_dinamico.filter(df_dinamico.resource.resourceType == "Encounter").select(
    F.col("resource.id").alias("encounter_id"),
    F.col("resource.subject.reference").alias("patient_reference"),
    F.col("resource.participant")[0].individual.reference.alias("practitioner_reference"),
    F.col("resource.serviceProvider.reference").alias("organization_reference"),
    F.col("resource.period.start").alias("encounter_start"),
    F.col("resource.period.end").alias("encounter_end")
)
encounters.show()

+--------------------+--------------------+----------------------+----------------------+--------------------+--------------------+
|        encounter_id|   patient_reference|practitioner_reference|organization_reference|     encounter_start|       encounter_end|
+--------------------+--------------------+----------------------+----------------------+--------------------+--------------------+
|2b772e4b-8e9f-438...|urn:uuid:0718123b...|  urn:uuid:0000016d...|  urn:uuid:78418eeb...|1935-09-30T09:04:...|1935-09-30T09:19:...|
|7c8a02ca-f312-433...|urn:uuid:0718123b...|  urn:uuid:0000016d...|  urn:uuid:4861d01f...|1941-12-28T09:04:...|1941-12-28T10:04:...|
|0b288f8e-f579-49b...|urn:uuid:0718123b...|  urn:uuid:0000016d...|  urn:uuid:4861d01f...|1945-07-31T10:04:...|1945-07-31T11:04:...|
|0b79390f-992c-4af...|urn:uuid:0718123b...|  urn:uuid:0000016d...|  urn:uuid:8881e3ee...|1945-07-31T10:04:...|1945-08-02T11:11:...|
|589adc6f-4329-49b...|urn:uuid:0718123b...|  urn:uuid:0000016d...|  urn:uuid

In [229]:
conditions =  df_dinamico.filter(df_dinamico.resource.resourceType == "Condition").select(
    F.col("resource.id").alias("condition_id"),
    F.col("resource.subject.reference").alias("patient_reference"),
    F.col("resource.encounter.reference").alias("encounter_reference"),
    F.col("resource.code.text").alias("condition_text"),
    F.col("resource.onsetDateTime").alias("onset_date")
)
conditions.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|        condition_id|   patient_reference| encounter_reference|      condition_text|          onset_date|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|e8724820-f81f-47a...|urn:uuid:0718123b...|urn:uuid:2b772e4b...|        Hypertension|1935-09-30T09:04:...|
|3284a52e-ba56-449...|urn:uuid:0718123b...|urn:uuid:0b288f8e...|        Appendicitis|1945-07-31T10:04:...|
|f1453845-969a-4d7...|urn:uuid:0718123b...|urn:uuid:0b79390f...|History of append...|1945-07-31T10:04:...|
|72bbb4ae-b523-4a1...|urn:uuid:0718123b...|urn:uuid:0b79390f...|         Prediabetes|1945-08-06T10:04:...|
|c1738686-6a96-499...|urn:uuid:0718123b...|urn:uuid:589adc6f...|   Anemia (disorder)|1945-10-15T09:04:...|
|ec78f74b-5518-4cc...|urn:uuid:0718123b...|urn:uuid:3ed700cf...|            Diabetes|1948-10-18T09:04:...|
|6c6497ce-8849-4d0...|urn:uuid:071812

In [233]:
devices =  df_dinamico.filter(df_dinamico.resource.resourceType == "Device").select(
    F.col("resource.id").alias("device_id"),
    F.col("resource.patient.reference").alias("patient_reference"),
    F.col("resource.type").alias("device_type"),
    F.col("resource.manufactureDate").alias("manufacture_date"),
    F.col("resource.expirationDate").alias("expiration_date")
)
# devices.show()



dynamic_schema_devices = spark.read.json(devices.rdd.map(lambda row: row.device_type)).schema
devices_json_exploded = devices.withColumn("exploded_device_type" , F.from_json(F.col("device_type"), dynamic_schema_devices)) # .select("exploded_name.*")
devices_json_exploded.show()



+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|           device_id|   patient_reference|         device_type|    manufacture_date|     expiration_date|exploded_device_type|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|86cc7603-0a80-445...|urn:uuid:df121e33...|{"coding":[{"syst...|2004-03-07T23:17:...|2029-03-23T00:17:...|{[{706004007, Imp...|
|ae974cb9-24bf-4ef...|urn:uuid:4b5e42a8...|{"coding":[{"syst...|2005-02-17T09:03:...|2030-03-04T09:03:...|{[{706004007, Imp...|
|ab1c2ee6-de75-47b...|urn:uuid:4cb0d703...|{"coding":[{"syst...|1975-11-13T09:03:...|2000-11-27T09:03:...|{[{72506001, Impl...|
|a141fa86-7edd-446...|urn:uuid:8055d1ca...|{"coding":[{"syst...|2018-09-10T22:53:...|2043-09-25T22:53:...|{[{706004007, Imp...|
|382c81b3-b589-411...|urn:uuid:86017b61...|{"coding":[{"syst...|2008-09-26T10:50:...|2033-10-11T10:50:..

In [236]:
df_medicamentos = df_dinamico_resource.where(df_dinamico_resource.resourceType == "MedicationRequest")

In [246]:
df_medicamentos['id', 'status', 'intent', 'medicationCodeableConcept.text', 'authoredOn'].show()

+--------------------+-------+------+--------------------+--------------------+
|                  id| status|intent|                text|          authoredOn|
+--------------------+-------+------+--------------------+--------------------+
|eb7f5891-2265-495...| active| order|Hydrochlorothiazi...|1935-09-30T09:04:...|
|0c07899f-1349-48a...| active| order|24 HR Metformin h...|1951-10-22T09:04:...|
|14a04af3-88be-404...|stopped| order|Naproxen sodium 2...|1953-07-28T10:04:...|
|d23a5043-3f64-4da...| active| order|insulin human, is...|1955-03-14T09:04:...|
|d5d91624-cd6c-46a...|stopped| order|   Simvistatin 10 MG|1965-09-06T10:04:...|
|0cffb54f-2956-43f...|stopped| order|   Simvistatin 10 MG|1966-09-06T10:04:...|
|b8792f87-354a-4b3...|stopped| order|   Simvistatin 10 MG|1967-09-06T10:04:...|
|a996c11c-6531-4ba...|stopped| order|   Simvistatin 10 MG|1968-09-05T10:04:...|
|f1809b0a-9979-41e...|stopped| order|   Simvistatin 10 MG|1969-09-05T10:04:...|
|3d71e455-b74b-408...|stopped| order|   

In [253]:
df_medicamentos = df_medicamentos[['id', 'status', 'intent', 'medicationCodeableConcept.text', 'authoredOn']]

In [247]:
# df_medicamentos.columns

In [ ]:
grava_tabela(conditions, "conditions")
grava_tabela(df_medicamentos, "medicamentos")
grava_tabela(patients_json, "sexo_paciente")
grava_tabela(organizations, "hospitais")
# grava_tabela(condicao_medica, "condicao_medica")
# grava_tabela(medicamentos, "medicamentos")
# grava_tabela(sexo_paciente, "sexo_paciente")

In [248]:
# encounters_with_patients = encounters.join(
#     patients_json,
#     encounters.patient_reference == patients_json.patient_id,
#     "inner"
# ).select(
#     encounters.encounter_id,
#     patients_json.given_name,
#     patients_json.family_name,
#     encounters.encounter_start,
#     encounters.encounter_end
# )

# # Exibe o resultado do join
# encounters_with_patients.show()

In [188]:
# claims_json_exploded.printSchema()
# claims_json_exploded.select('exploded_claim_total_value.value').show()
# claims.withColumn('value', F.col(claims_json_exploded.select('exploded_claim_total_value.value'))) # .withColumn('currency', claims_json_exploded.select('exploded_claim_total_value.currency')).show()

In [200]:
# devices = df_dinamico.filter(df_dinamico.resource.resourceType == "Device").select(
#     F.col("resource.id").alias("device_id"),
#     F.col("resource.patient.reference").alias("patient_reference"),
#     F.col("resource.type.text").alias("device_type"),
#     F.col("resource.manufactureDate").alias("manufacture_date"),
#     F.col("resource.expirationDate").alias("expiration_date")
# )

In [ ]:
# organizations = df_dinamico.filter(col("resource.resourceType") == "Organization").select(
#     col("resource.id").alias("organization_id"),
#     col("resource.name").alias("organization_name"),
#     col("resource.address")[0].city.alias("city"),
#     col("resource.address")[0].state.alias("state"),
#     col("resource.address")[0].country.alias("country")
# )

# practitioners = df_dinamico.filter(col("resource.resourceType") == "Practitioner").select(
#     col("resource.id").alias("practitioner_id"),
#     col("resource.name")[0].family.alias("practitioner_family_name"),
#     col("resource.name")[0].given[0].alias("practitioner_given_name"),
#     col("resource.gender").alias("practitioner_gender")
# )

# encounters = df_dinamico.filter(col("resource.resourceType") == "Encounter").select(
#     col("resource.id").alias("encounter_id"),
#     col("resource.subject.reference").alias("patient_reference"),
#     col("resource.participant")[0].individual.reference.alias("practitioner_reference"),
#     col("resource.serviceProvider.reference").alias("organization_reference"),
#     col("resource.period.start").alias("encounter_start"),
#     col("resource.period.end").alias("encounter_end")
# )

# conditions = df_dinamico.filter(col("resource.resourceType") == "Condition").select(
#     col("resource.id").alias("condition_id"),
#     col("resource.subject.reference").alias("patient_reference"),
#     col("resource.encounter.reference").alias("encounter_reference"),
#     col("resource.code.text").alias("condition_text"),
#     col("resource.onsetDateTime").alias("onset_date")
# )

# devices = df_dinamico.filter(col("resource.resourceType") == "Device").select(
#     col("resource.id").alias("device_id"),
#     col("resource.patient.reference").alias("patient_reference"),
#     col("resource.type.text").alias("device_type"),
#     col("resource.manufactureDate").alias("manufacture_date"),
#     col("resource.expirationDate").alias("expiration_date")
# )

# # claims = df_dinamico.filter(F.col("resource.resourceType") == "Claim").select(
# claims = df_dinamico.filter(df_dinamico.resource.resourceType == "Claim").select(
#     F.col("resource.id").alias("claim_id"),
#     F.col("resource.patient.reference").alias("patient_reference"),
#     F.col("resource.provider.reference").alias("organization_reference"),
#     F.col("resource.created").alias("claim_created"),
    
#     F.col("resource.total").alias("claim_total_value"),
#     # F.col("resource.total.currency").alias("claim_currency")
# )


# patients.show()
# organizations.show()
# practitioners.show()
# encounters.show()
# conditions.show()
# devices.show()
# claims.show()


# encounters_with_patients = encounters.join(
#     patients,
#     encounters.patient_reference == patients.patient_id,
#     "inner"
# ).select(
#     encounters.encounter_id,
#     patients.given_name,
#     patients.family_name,
#     encounters.encounter_start,
#     encounters.encounter_end
# )

# # Exibe o resultado do join
# encounters_with_patients.show()